In [1]:
import requests
import pandas as pd

from bs4 import BeautifulSoup
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [2]:
csv_name = 'target_protein_wiki'
df = pd.read_csv(csv_name + '.csv')

In [3]:
def get_wiki_content(url):
    """ 주어진 Wikipedia URL에서 내용을 크롤링하여 텍스트를 반환합니다. """
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        paragraphs = soup.find_all('p')
        content = ' '.join([para.text for para in paragraphs])
        return content.strip()
    except Exception as e:
        return f"Error: {str(e)}"

In [4]:
def summarize_text(text, llm_chain):
    """주어진 텍스트를 LLM을 통해 요약합니다."""
    try:
        return llm_chain.invoke(text).content
    except Exception as e:
        return f"Error during summarization: {str(e)}"

In [5]:
# LLM 설정
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Prompt 설정
prompt = PromptTemplate(
    input_variables=["text"],  # LLM이 처리할 입력 변수 이름
    template="Please summarize the following text: {text}"  # LLM에게 줄 실제 프롬프트
)

# LLMChain 생성
llm_chain = prompt | llm

In [6]:
# "Wiki_documents" 컬럼 생성 및 요약 처리
df['Wiki_documents'] = df['Wiki'].apply(
    lambda x: None if x == "FALSE" else summarize_text(get_wiki_content(x), llm_chain)
)

In [8]:
# 결과를 새 CSV 파일로 저장
output_file_path = csv_name + '_wiki_documents_summarized.csv'
df.to_csv(output_file_path, index=False)
print(f"새 파일이 저장되었습니다: {output_file_path}")

새 파일이 저장되었습니다: target_protein_wiki_wiki_documents_summarized.csv


In [ ]:
# import requests
# from bs4 import BeautifulSoup
# import pandas as pd

# csv_name = 'target_protein_wiki'
# df = pd.read_csv(csv_name + '.csv')

# def get_wiki_content(url):
#     """ 주어진 Wikipedia URL에서 내용을 크롤링하여 텍스트를 반환합니다. """
#     try:
#         response = requests.get(url)
#         soup = BeautifulSoup(response.text, 'html.parser')
#         paragraphs = soup.find_all('p')
#         content = ' '.join([para.text for para in paragraphs])
#         return content.strip()
#     except Exception as e:
#         return f"Error: {str(e)}"

# # 새로운 컬럼 "Wiki_documents" 생성
# df['Wiki_documents'] = df['Wiki'].apply(lambda x: None if x == "FALSE" else get_wiki_content(x))

# # 결과를 새 CSV 파일로 저장
# output_file_path = csv_name + '_wiki_documents.csv'
# df.to_csv(output_file_path, index=False)

# print(f"새 파일이 저장되었습니다: {output_file_path}")
# None GPT